# Colour prediction for a MTG card image

---

Import necessary libraries

In [0]:
# !sudo pip install h5py
import h5py
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from scipy import misc
import glob
from google.colab import drive
import pickle
import pandas as pd
from sklearn import preprocessing

Mount Google drive

In [0]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


Unzip the data

In [0]:
!unzip /content/gdrive/'My Drive'/'Magic Final Project'/data/images/cropped.zip

Archive:  /content/gdrive/My Drive/Magic Final Project/data/images/cropped.zip
   creating: cropped/
  inflating: cropped/39934090-36a6-4183-9176-97ea932d2685.jpg  
  inflating: cropped/688a8de2-0167-4b35-a38d-3574034a892c.jpg  
  inflating: cropped/724fd1ea-05ad-497c-9989-825ada48e684.jpg  
  inflating: cropped/ca31ce70-24c5-4fb2-8d88-c9ffa8474c8f.jpg  
  inflating: cropped/02a3062e-8b83-4ee4-8139-8eee84df37fe.jpg  
  inflating: cropped/45704604-a1b3-4225-8466-aa76136c84a8.jpg  
  inflating: cropped/c3739d37-7865-46bf-abbc-7a5678709508.jpg  
  inflating: cropped/44814da0-3bc8-423d-9b22-3fea123048fa.jpg  
  inflating: cropped/96b5362a-bbca-4dc3-a320-3664609fe169.jpg  
  inflating: cropped/f7fc5b67-f521-4ba4-a10f-103e8b6af688.jpg  
  inflating: cropped/93311f80-0d0e-4005-ba43-5dbfe438d127.jpg  
  inflating: cropped/83585337-56a9-44d2-9ed1-8a959bcfb010.jpg  
  inflating: cropped/f48fda3d-5e6f-4f32-8fca-030521f6df98.jpg  
  inflating: cropped/cba5fb67-e161-4e89-be3e-c8021122ff19.jpg  
  i

Data Preparation

In [0]:
df = pd.read_csv("gdrive/My Drive/Magic Final Project/data/cards.csv")
df = df.sort_values('Unnamed: 0')
df = df.reset_index(drop=True)
df1 = pd.DataFrame(df['colors'][10000:20000])
mask = (df1['colors'].str.len() == 1)
df1 = df1.loc[mask]
sgl_clr_exists= np.array(df1.index)
le = preprocessing.LabelEncoder()
le.fit(df1['colors'])
df1['color_code'] = le.transform(df1['colors'])
#to get inverse of color code - use le.inverse_transform(df1['color_code'])

In [0]:
image_names = list(df.iloc[sgl_clr_exists,:]['Unnamed: 0'])
image_labels = list(df1['color_code'])

In [0]:
len(set(image_labels))

5

In [0]:
print(len(image_names), len(image_labels))

7051 7051


Reading Training Images

In [0]:
data = np.zeros((len(image_names), 224, 224, 3), dtype='float16')
for ind, name in enumerate(image_names):
    img = "cropped/"+name+".jpg"
    data[ind] = np.array(Image.open(img).resize((224,224)))

In [0]:
from keras.utils import to_categorical
data_labels = to_categorical(np.array(image_labels))

In [0]:
data_labels.shape

(7051, 5)

In [0]:
from keras.models import Sequential,Model,load_model
from keras.optimizers import SGD
from keras.layers import BatchNormalization, Lambda, Input, Dense, Convolution2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Dropout, Flatten, merge, Reshape, Activation
from keras.layers.merge import Concatenate
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
import numpy as np
import keras.backend as K

Reference: https://github.com/beerboaa/Color-Classification-CNN/blob/master/color_net_training.ipynb
Paper: https://arxiv.org/pdf/1510.07391.pdf

In [0]:
def color_net(num_classes):
    # placeholder for input image
    input_image = Input(shape=(224,224,3))
    # ============================================= TOP BRANCH ===================================================
    # first top convolution layer
    top_conv1 = Convolution2D(filters=48,kernel_size=(11,11),strides=(4,4),
                              input_shape=(224,224,3),activation='relu')(input_image)
    top_conv1 = BatchNormalization()(top_conv1)
    top_conv1 = MaxPooling2D(pool_size=(3,3),strides=(2,2))(top_conv1)

    # second top convolution layer
    # split feature map by half
    top_top_conv2 = Lambda(lambda x : x[:,:,:,:24])(top_conv1)
    top_bot_conv2 = Lambda(lambda x : x[:,:,:,24:])(top_conv1)

    top_top_conv2 = Convolution2D(filters=64,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(top_top_conv2)
    top_top_conv2 = BatchNormalization()(top_top_conv2)
    top_top_conv2 = MaxPooling2D(pool_size=(3,3),strides=(2,2))(top_top_conv2)

    top_bot_conv2 = Convolution2D(filters=64,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(top_bot_conv2)
    top_bot_conv2 = BatchNormalization()(top_bot_conv2)
    top_bot_conv2 = MaxPooling2D(pool_size=(3,3),strides=(2,2))(top_bot_conv2)

    # third top convolution layer
    # concat 2 feature map
    top_conv3 = Concatenate()([top_top_conv2,top_bot_conv2])
    top_conv3 = Convolution2D(filters=192,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(top_conv3)

    # fourth top convolution layer
    # split feature map by half
    top_top_conv4 = Lambda(lambda x : x[:,:,:,:96])(top_conv3)
    top_bot_conv4 = Lambda(lambda x : x[:,:,:,96:])(top_conv3)

    top_top_conv4 = Convolution2D(filters=96,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(top_top_conv4)
    top_bot_conv4 = Convolution2D(filters=96,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(top_bot_conv4)

    # fifth top convolution layer
    top_top_conv5 = Convolution2D(filters=64,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(top_top_conv4)
    top_top_conv5 = MaxPooling2D(pool_size=(3,3),strides=(2,2))(top_top_conv5) 

    top_bot_conv5 = Convolution2D(filters=64,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(top_bot_conv4)
    top_bot_conv5 = MaxPooling2D(pool_size=(3,3),strides=(2,2))(top_bot_conv5)

    # ============================================= TOP BOTTOM ===================================================
    # first bottom convolution layer
    bottom_conv1 = Convolution2D(filters=48,kernel_size=(11,11),strides=(4,4),
                              input_shape=(227,227,3),activation='relu')(input_image)
    bottom_conv1 = BatchNormalization()(bottom_conv1)
    bottom_conv1 = MaxPooling2D(pool_size=(3,3),strides=(2,2))(bottom_conv1)

    # second bottom convolution layer
    # split feature map by half
    bottom_top_conv2 = Lambda(lambda x : x[:,:,:,:24])(bottom_conv1)
    bottom_bot_conv2 = Lambda(lambda x : x[:,:,:,24:])(bottom_conv1)

    bottom_top_conv2 = Convolution2D(filters=64,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(bottom_top_conv2)
    bottom_top_conv2 = BatchNormalization()(bottom_top_conv2)
    bottom_top_conv2 = MaxPooling2D(pool_size=(3,3),strides=(2,2))(bottom_top_conv2)

    bottom_bot_conv2 = Convolution2D(filters=64,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(bottom_bot_conv2)
    bottom_bot_conv2 = BatchNormalization()(bottom_bot_conv2)
    bottom_bot_conv2 = MaxPooling2D(pool_size=(3,3),strides=(2,2))(bottom_bot_conv2)

    # third bottom convolution layer
    # concat 2 feature map
    bottom_conv3 = Concatenate()([bottom_top_conv2,bottom_bot_conv2])
    bottom_conv3 = Convolution2D(filters=192,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(bottom_conv3)

    # fourth bottom convolution layer
    # split feature map by half
    bottom_top_conv4 = Lambda(lambda x : x[:,:,:,:96])(bottom_conv3)
    bottom_bot_conv4 = Lambda(lambda x : x[:,:,:,96:])(bottom_conv3)

    bottom_top_conv4 = Convolution2D(filters=96,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(bottom_top_conv4)
    bottom_bot_conv4 = Convolution2D(filters=96,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(bottom_bot_conv4)

    # fifth bottom convolution layer
    bottom_top_conv5 = Convolution2D(filters=64,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(bottom_top_conv4)
    bottom_top_conv5 = MaxPooling2D(pool_size=(3,3),strides=(2,2))(bottom_top_conv5) 

    bottom_bot_conv5 = Convolution2D(filters=64,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(bottom_bot_conv4)
    bottom_bot_conv5 = MaxPooling2D(pool_size=(3,3),strides=(2,2))(bottom_bot_conv5)

    # ======================================== CONCATENATE TOP AND BOTTOM BRANCH =================================
    conv_output = Concatenate()([top_top_conv5,top_bot_conv5,bottom_top_conv5,bottom_bot_conv5])

    # Flatten
    flatten = Flatten()(conv_output)

    # Fully-connected layer
    FC_1 = Dense(units=4096, activation='relu')(flatten)
    FC_1 = Dropout(0.6)(FC_1)
    FC_2 = Dense(units=4096, activation='relu')(FC_1)
    FC_2 = Dropout(0.6)(FC_2)
    output = Dense(units=num_classes, activation='softmax')(FC_2)
    
    model = Model(inputs=input_image,outputs=output)
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    # sgd = SGD(lr=0.01, momentum=0.9, decay=0.0005, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [0]:
num_classes = len(set(image_labels))
# batch_size = 32
# nb_epoch = 5

# initialise model
model = color_net(num_classes)

from keras.callbacks import EarlyStopping

early_stopping = [EarlyStopping(monitor='val_acc',
                          min_delta=0,
                          patience=5,
                          verbose=2, mode='auto')]

model.fit(data, data_labels, validation_split = 0.2, verbose=1,
                callbacks=early_stopping, batch_size=64, epochs=100)

# model.fit(data, data_labels, validation_split = 0.2, batch_size=32, epochs=10)

Train on 5640 samples, validate on 1411 samples
Epoch 1/100
5640/5640 [==============================] - 12s 2ms/step - loss: 1.7127 - acc: 0.2899 - val_loss: 1.4491 - val_acc: 0.4040
Epoch 2/100
5640/5640 [==============================] - 10s 2ms/step - loss: 1.4636 - acc: 0.3835 - val_loss: 1.4188 - val_acc: 0.4004
Epoch 3/100
5640/5640 [==============================] - 10s 2ms/step - loss: 1.4247 - acc: 0.4094 - val_loss: 1.4346 - val_acc: 0.4047
Epoch 4/100
5640/5640 [==============================] - 9s 2ms/step - loss: 1.4011 - acc: 0.4215 - val_loss: 1.4291 - val_acc: 0.4125
Epoch 5/100
5640/5640 [==============================] - 9s 2ms/step - loss: 1.3910 - acc: 0.4316 - val_loss: 1.4037 - val_acc: 0.4089
Epoch 6/100
5640/5640 [==============================] - 9s 2ms/step - loss: 1.3791 - acc: 0.4362 - val_loss: 1.4521 - val_acc: 0.3955
Epoch 7/100
5640/5640 [==============================] - 9s 2ms/step - loss: 1.3656 - acc: 0.4411 - val_loss: 1.3862 - val_acc: 0.4352
Epoc

In [0]:
df1 = pd.DataFrame(df['colors'][20000:])
mask = (df1['colors'].str.len() == 1)
df1 = df1.loc[mask]
sgl_clr_exists= np.array(df1.index)
# le = preprocessing.LabelEncoder()
# le.fit(df1['colors'])
df1['color_code'] = le.transform(df1['colors'])
test_image_names = list(df.iloc[sgl_clr_exists,:]['Unnamed: 0'])
test_image_labels = list(df1['color_code'])

In [0]:
len(test_image_names), len(test_image_labels)

(3256, 3256)

In [0]:
test_data = np.zeros((len(test_image_names), 224, 224, 3), dtype='float16')
for ind, name in enumerate(test_image_names):
    img = "cropped/"+name+".jpg"
    test_data[ind] = np.array(Image.open(img).resize((224,224)))

In [0]:
test_data_labels = to_categorical(np.array(test_image_labels))

test_data.shape, test_data_labels.shape

((3256, 224, 224, 3), (3256, 5))

In [0]:
model.evaluate(test_data, test_data_labels)

3256/3256 [==============================] - 3s 1ms/step


[1.3591366962367253, 0.44471744464422036]